# How to classify new sequences?

This notebook contains a tutorial that shows you how to classify new sequences.

### Input data
* Camera trap images, in folders per deployment
* Agouti export files: observations, assets and pickup-setup

### Load the predefined configuration file

This file contains the paths to the different folders. <br>
If you have the same data structure as in this repository, you can use the predefined configuration file. If needed, you can change the paths according to you own folder structure.

The following paths are defined:
* **general_folder_path** : orginal camera trap images and the Agouti export files (assets, observations and pickup-setup)
* **resized_folder_path** : resized camera trap images
* **preprocessing_output_path** : preprocessing output
* **crop_output_path** : cropped images (optional)
* **draw_output_path** : regions of interest indicated on original camera trap images (optional)
* **bottleneck_features_output_path** : extracted bottleneck features
* **weight_path** : weights top model
* **predictions_output_path** : predictions

In [7]:
from yaml import load

with open("config.yml") as yaml_config:
    config = load(yaml_config)

### Check if all folders exist and create folder if needed

In [6]:
import os

for path in config:
    if not os.path.exists(config[path]):
        os.makedirs(config[path])

### Step 1: resize images
The camera trap images are resized to 50% of their original size. This strongly decreases the computational time, while the performance remains the same.<br>

Input: original camera trap images and Agouti export file (observations)<br>
Output: resized camera trap images in a similar folder structure as the original images

In [ ]:
from Preprocessing.Resize_images import resize_images

resize_images(config["general_folder_path"], 
              config["resized_folder_path"])

### Step 2: preprocess images
During the preprocessing, the regions of interest in the images are determined. All images of a sequence are used to construct a background image. Subsequently, the regions of interest in a camera trap image are determined by computing the difference between this background image and the camera trap image.

Input: resized camera trap images and Agouti export files(observations + assets + pickupsetup)<br>
Output: cvs-file containing the coordinates of the regions of interest in every camera trap image

In [ ]:
from Preprocessing.Preprocessing import preprocessing

preprocessing(config["general_folder_path"], 
              config["resized_folder_path"], 
              config["preprocessing_output_path"])

#### Optional: crop images or indicate regions of interest
We can crop the camera trap images or indicate the regions of interest on the camera trap images to see the result of the preprocessing. <br>
This step is optional and not required to classify the images.

In [ ]:
from Preprocessing.Crop_images import crop_images
from Preprocessing.Draw_boxes import draw_boxes

crop_images(config["preprocessing_output_path"], config["resized_folder_path"], config["crop_output_path"])
draw_boxes(config["preprocessing_output_path"], config["resized_folder_path"], config["draw_output_path"])

In [ ]:
from PIL import image

# Show a camera trap image with indication of the regions of interest
deployment = os.listdir(config["draw_output_path"])[0]
image_name = os.listdir(deployment)[0]
image = Image.open(os.path.join(config["draw_output_path"],deployment, image_name))
image

### Step 3: extract bottleneck features

The pretrained convolutional neural network ResNet50 is used to convert the images to bottleneck features.

Input: resized camera trap images and preprocessing output containing the coordinates of the boxes <br>
Output: bottleneck features

In [ ]:
from Network.ResNet50_Bottleneck_features_Predict import extract_bottleneck_features

extract_bottleneck_features(config["preprocessing_output_path"], 
                            config["bottleneck_features_output_path"], 
                            config["resized_folder_path"])

### Step 4 : run top model to classify the new images

The extracted bottleneck features are fed to the new top model to predict the labels of the new images.

Input: extracted bottleneck features <br>
Ouput: probabilities of the output classes

In [ ]:
from Network.ResNet50_Hierarchical_Bottleneck_Predict import hierarchical_bottleneck_predict

hierarchical_bottleneck_predict(config["bottleneck_features_output_path"], 
                                config["weight_path"], 
                                config["predictions_output_path"])

### Step 5 : convert output probabilities to hierarchical classification

The predictions of the individual images are aggregated to a hierarchical prediction for every sequence.

Input: probabilities of the output classes for the individual images <br>
Output: hierarchical classification of the sequences

In [ ]:
from Network.Hierachical_processing_predictions import hierarchical_predictions_sequences

hierarchical_predictions_sequences( config["predictions_output_path"], 
                                   config["bottleneck_features_output_path"])

We can load the file containing the hierarchical predictions to see the predicted labels.

In [ ]:
import pandas as pd

predictions = pd.read_csv(os.path.join(config["predictions_output_path"],'hierarchical_predictions_sequences.csv'), sep = ';')
predictions.drop(['level_1_p','level_2_p','level_3_p','level_4_p','level_5_p'], axis=1, inplace=True)

print(predictions)

### Optional: Object localization

The class activation maps can be used to localize the objects in the cropped camera trap images. <br>
Since this step uses the cropped images, make sure to first run the optional cropping step above.

In [ ]:
from Network.ResNet_CAM import object_localization

img_path = os.path.join(config["crop_output_path"], os.listdir(config["crop_output_path"])[0])
object_localization(img_path)